# Downloading Euler's Publications

## Makes usable screen bigger

In [26]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Packages & definitions

In [27]:
# Necessary packages:
from urllib.request import urlretrieve
import re
import os
import pandas as pd
import numpy as np
import time
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 1000)

# If not yet installed, install these packages now:
try:
    from PyPDF2 import PdfFileReader as pr
except ImportError:
    os.system('python -m pip install PyPDF2')
    from PyPDF2 import PdfFileReader as pr

In [28]:
def tryOpenPDF(pdffilename):
    try:
        file = open(pdffilename, 'rb')
        read = pr(file)
        successful = True
    except:
        successful = False
    return successful

In [29]:
def downloadPub(obj, sleep=2):
    
    if obj['URLpdf']:
        pdfurl = obj['URLpdf']
        if obj['Enestrom Number']:
            pdffilename = str(int(obj['Enestrom Number'])) + '-' + re.sub('[#%&{}\\<>*?/$!\'\":@+`|=]', '', obj['Title']).replace(' ', '_')[:22] + '-' + str(obj['Published Date'])
        else:
            pdffilename = re.sub('[#%&{}\\<>*?/$!\'\":@+`|=]', '', obj['Title']).replace(' ', '_')[:22] + '-' + str(obj['Published Date'])
        pdfpath = './data/pdf/' + pdffilename + '.pdf'
        
        # Check length of not existing pdf list
        dfMissingPDF = makePDFfileList(dfEulerPdf)
        notdownloaded = len(dfMissingPDF)

        # Make folders if necessary
        if not os.path.exists('./data/pdf'):
            os.mkdir('./data/pdf')

        # Test if file exists:
        if os.path.isfile(pdfpath):
            print('file {} exists, passing'.format(pdffilename))
        else:
            # Download
            try:
                urlretrieve(pdfurl, pdfpath)
            except:
                print('trouble downloading!')
            # Test if file can be opened
            try:
                successful = tryOpenPDF(pdfpath)
                if not successful:
                    os.remove(pdfpath)
                    dfMissingPDF = makePDFfileList(dfEulerPdf)
                    notyetdownloaded = len(dfMissingPDF)
                    if notyetdownloaded < notdownloaded:
                        downloadPub(obj, sleep=2)
                    else:
                        print('No PDF variable under {}'.format(pdfurl))
                else:
                    print("Successfully downloaded {} to {}".format(pdfurl, pdffilename + '.pdf'))
            except:
                print('trouble opening!')
            if sleep:
                time.sleep(sleep)
    else:
        print("The publication {} has no PDF URL!".format(pdffilename))

In [30]:
def makePDFfileList(df):
    pdf_file_list = []
    rootDirpdf = './data/pdf/'

    for (paths, dirs, files) in os.walk(rootDirpdf, topdown=False):
        for file in files:
            pdf_file_list.append(os.path.join(paths, file))
            
    pdf_file_list = [i for i in pdf_file_list if i.endswith('.pdf') if i.find('/SYNOFILE_THUMB') == -1 if i.find('.ipynb_checkpoints') == -1 if not re.match('@eaDir/|@SynoResource|@SynoEAStream',i)]
    dfPDF = pd.DataFrame({'pdfpath': pdf_file_list, 'pathPDF': pdf_file_list})
    dfPDF = df.merge(dfPDF, on = 'pdfpath', how = 'outer').fillna('None').reset_index(drop = True)
    dfMissingPDF = dfPDF[dfPDF.pathPDF == 'None'].reset_index(drop = True)
    return dfMissingPDF

## Loading dataframe

In [31]:
dfEuler = pd.read_csv('./data/dfEulerPub_v1.csv', sep = ';')

In [32]:
dfEuler['pdfpath'] = ['./data/pdf/' + str(int(dfEuler['Enestrom Number'][i])) + '-' + re.sub('[#%&{}\\<>*?/$!\'\":@+`|=]', '', dfEuler['Title'][i]).replace(' ', '_')[:22] + '-' + str(dfEuler['Published Date'][i]) + '.pdf' if pd.notna(dfEuler.URLpdf[i]) else np.nan for i in range(len(dfEuler))]

In [33]:
dfEulerPdf = dfEuler[dfEuler.URLpdf.notnull()].reset_index()

In [34]:
dfMissingPDF = makePDFfileList(dfEulerPdf)

In [38]:
print('Publications of Euler with PDF: ', len(dfEulerPdf))
print('Missing PDFs: ', len(dfMissingPDF))

Publications of Euler with PDF:  838
Missing PDFs:  0


## Downloading the publications

In [25]:
for i in range(len(dfMissingPDF)):
    downloadPub(dfMissingPDF.iloc[i])

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1017&context=euler-works to 18-De_Indorum_anno_solari-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1018&context=euler-works to 19-De_progressionibus_tra-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1019&context=euler-works to 20-De_summatione_innumera-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1020&context=euler-works to 21-Quomodo_data_quacunque-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1021&context=euler-works to 22-De_communicatione_motu-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1022&context=euler-works to 23-De_curvis_rectificabil-1738.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1077&context=euler-works to 78-Dissertation_sur_la_me-1745.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1078&context=euler-works to 79-Problema_geometricum_p-1745.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1081&context=euler-works to 82-De_la_force_de_percuss-1746.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1082&context=euler-works to 83-Sur_quelques_proprieté-1746.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1083&context=euler-works to 84-Leonhardi_Euleri_anima-1746.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1084&context=euler-works to 85-Solutio_problematis_ca-1746.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1134&context=euler-works to 135-Variae_demostrationes_-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1135&context=euler-works to 136-De_propagatione_pulsuu-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1136&context=euler-works to 137-Examen_artificii_navis-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1137&context=euler-works to 138-De_motu_nodorum_lunae_-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1138&context=euler-works to 139-Quantum_motus_terrae_a-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1139&context=euler-works to 140-Sur_la_vibration_des_c-1750.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1190&context=euler-works to 191-De_partitione_numeroru-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1191&context=euler-works to 192-Solutio_problematis_ge-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1192&context=euler-works to 193-De_perturbatione_motus-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1193&context=euler-works to 194-De_machinis_in_genere-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1194&context=euler-works to 195-De_motu_tautochrono_pe-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1195&context=euler-works to 196-Emendatio_laternae_mag-1753.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?ar

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1246&context=euler-works to 247-De_seriebus_divergenti-1760.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1247&context=euler-works to 248-De_cochlea_Archimedis-1760.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1248&context=euler-works to 249-De_aptissima_figura_ro-1760.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1249&context=euler-works to 250-Methodus_inveniendi_in-1761.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1250&context=euler-works to 251-De_integratione_aequat-1761.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1251&context=euler-works to 252-Observationes_de_compa-1761.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?ar

...

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1634&context=euler-works to 635-Innumera_theoremata_ci-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1635&context=euler-works to 636-De_multiplicatione_ang-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1636&context=euler-works to 637-Nova_demonstratio,_quo-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1637&context=euler-works to 638-De_innumeris_curvis_al-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1638&context=euler-works to 639-De_innumeris_curvis_al-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1639&context=euler-works to 640-Comparatio_valorum_for-1789.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1688&context=euler-works to 689-Integratio_formulae_di-1795.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1689&context=euler-works to 690-Evolutio_formulae_inte-1795.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1690&context=euler-works to 691-Problema_geometricum_q-1795.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1691&context=euler-works to 692-Solutio_problematis_ma-1795.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1692&context=euler-works to 693-De_centro_simulitudini-1795.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1693&context=euler-works to 694-Ulterior_disquitio_de_-1797.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1742&context=euler-works to 743-De_serie_maxime_memora-1813.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1743&context=euler-works to 744-De_divisoribus_numeror-1815.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1744&context=euler-works to 745-De_fractionibus_contin-1815.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1745&context=euler-works to 746-Methodus_succincta_sum-1815.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1746&context=euler-works to 747-De_seriebus_memorabili-1815.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1747&context=euler-works to 748-Investigatio_quadrilat-1815.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1798&context=euler-works to 799-Fragmenta_commentation-1849.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1799&context=euler-works to 800-Brief_von_L._Euler_an_-1851.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1800&context=euler-works to 801-Drei_Briefe_von_L._Eul-1852.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1801&context=euler-works to 802-Einige_Zeilen_aus_Brie-1853.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1802&context=euler-works to 803-Vierzehn_Briefe_oder_A-1854.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1803&context=euler-works to 804-Zwei_Briefe_von_L._Eul-1860.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a

Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1853&context=euler-works to 854-Différentes_pièces_sur-1862.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1854&context=euler-works to 855-Principia_pro_motu_san-1862.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1855&context=euler-works to 856-Fragmentum_ex_adversar-1862.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1856&context=euler-works to 857-Auszüge_aus_Briefen_vo-1880.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1857&context=euler-works to 858-Lettres_inédites_dEule-1886.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?article=1859&context=euler-works to 860-Fourteen_letters_from_-1896.pdf
Successfully downloaded https://scholarlycommons.pacific.edu/cgi/viewcontent.cgi?a